In [5]:
#mosaic an image into a single column and
#save the mosaic
import os.path as path
from shutil import copyfile
import cv2
from opencvlib import view
from opencvlib.view import draw_str
import opencvlib
import opencvlib.imgpipes.generators as gen
import funclib.iolib as iolib
import opencvlib.transforms as tf
import opencvlib.view as view
from funclib.iolib import PrintProgress as PP
from funclib.iolib import PrintProgressFlash as PPF

print('Done')

Done


In [11]:
#DENOISE SINGLE


#h : parameter deciding filter strength. Higher h value removes noise better, but removes details of image also. (10 is ok)
#hForColorComponents : same as h, but for color images only. (normally same as h)
#templateWindowSize : should be odd. (recommended 7)
#searchWindowSize : should be odd. (recommended 21)
h = 12          
I = cv2.imread(r'C:\Users\Graham Monkman\OneDrive\ebay\1 tolist\gamepad_dragon\DSCF1545.JPG',cv2.IMREAD_UNCHANGED)
I = cv2.fastNlMeansDenoisingColored(I, None, h, h, 7, 21)
cv2.imwrite(r'C:\Users\Graham Monkman\OneDrive\ebay\1 tolist\gamepad_dragon\DSCF1545_DN.JPG', I)            

True

In [ ]:
#easy_mosaic
#This should resize images to w=1920, overwriting the originals
#and then create 2x2 mosaics for each image, and delete the originals
#used in the mosaic

cnt = 0

PATCHES = 4
COLS = 2
DO_DELETE = True #delete the original images
DO_RESIZE = False
DO_DENOISE = False
CREATE_BACKUP = True #create a backup
MOSAIC = True

fld_nr = 1

FOLDERS = [
r'C:\Users\Graham Monkman\OneDrive\ebay\1 tolist\thinkstation_mobos'
]


for FLD in FOLDERS:
    FLD = path.normpath(FLD)
    FLD_BACKUP = path.normpath('%s/_backup' % FLD)
    WIDTH = 2400 #1920
    INCLUDE = ['DSCF','dscf'] #only include images which match this wildcard

    if CREATE_BACKUP:
        iolib.create_folder(FLD_BACKUP)
        fp = gen.FromPaths(FLD)
        for _, fname, _ in fp.generate(pathonly=True):
            d, f, e = iolib.get_file_parts2(fname)
            dst = '%s/%s' % (FLD_BACKUP, f)
            copyfile(fname, dst)
        print('Copied files to %s' % FLD_BACKUP)


    def resize_(I, fname, width=WIDTH, overwrite_orig=False):
        '''overwrite the original image'''
        I = tf.resize(image=I, width=width)
        
        if overwrite_orig:
            cv2.imwrite(fname, I)
        return I

    if DO_RESIZE or DO_DENOISE:
        print('\n\nResizing/Denoising')
        j = 1
        fp = gen.FromPaths(FLD)
        n = len(list(1 for _ in fp.generate(pathonly=True)))
        P = PP(maximum=n)
        for I, fname, _ in fp.generate():
            _, f, ext = iolib.get_file_parts(fname)
            if DO_RESIZE:
                I = resize_(I, fname, overwrite_orig=False)

            if DO_DENOISE:
                I = cv2.fastNlMeansDenoisingColored(I, None, 12, 12, 11, 31)

            cv2.imwrite(fname, I)
            P.increment(suffix='1 of %s' % j)
            j+=1



    if MOSAIC:
    
        print('\n\nMosaicing')
        files = [f for f in iolib.file_list_generator1(FLD, ['.jpg'], recurse=False)]
        images = []
        pf = PPF()
        for i, f in enumerate(files):
            if any([s in f for s in INCLUDE]):
                images.append(f)
                if len(images) == PATCHES:
                    I = view.mosaic(images, cols=COLS, save_as='%s/mosaic%s.jpg' % (FLD, int((i + 1)/4)))
                    pf.update()
                    if DO_DELETE:
                        iolib.files_delete2(images)
                    images = []
            

        if len(images) > 1:
            I = view.mosaic(images, cols=COLS, save_as='%s/mosaic%s.jpg' % (FLD, '_last'))
            pf.update()
            if DO_DELETE:
                iolib.files_delete2(images)
            images = []
    

print('Done')

Copied files to C:\Users\Graham Monkman\OneDrive\ebay\1 tolist\thinkstation_mobos\_backup


Mosaicing




C:\development\python\opencvlib\view.py:321: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  out = _np.vstack(map(_np.hstack, rows))


In [10]:
from funclib.iolib import PrintProgressFlash as PPF